In [1]:
!pip install python-socketio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
# @title Ostrich Legs Worker Node (v4.0 - Adaptive Benchmark & Polling)
# @markdown Run this cell to join the compute swarm!

import sys
import subprocess
import time
import json
import os
import uuid
import threading
import platform
import numpy as np

# --- 1. INSTALL DEPENDENCIES ---
try:
    import socketio
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-socketio[client]", "requests"])
    import socketio


In [ ]:
# Try importing Torch for GPU support
HAS_GPU = False
try:
    import torch
    if torch.cuda.is_available():
        HAS_GPU = True
        print(f"🚀 GPU DETECTED: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️ GPU not found. Falling back to CPU.")
except ImportError:
    print("⚠️ PyTorch not installed. Falling back to CPU.")


In [ ]:
# --- 4. KERNELS ---

def run_stress_test(iterations):
    """CPU Bound Stress Test"""
    start = time.time()
    count = int(iterations or 100000)
    # CPU heavy vector math
    x = np.random.rand(int(count/100))
    np.sin(x) * np.sqrt(x)
    return float(time.time() - start)

def run_matrix_mul(data):
    """Hybrid GPU/CPU Matrix Multiplication"""
    start = time.time()

    # Extract dimensions (default 300 for normal jobs)
    size = int(data.get('size', 300))

    if HAS_GPU:
        # GPU PATH
        a = torch.rand(size, size, device='cuda')
        b = torch.rand(size, size, device='cuda')
        torch.matmul(a, b)
        torch.cuda.synchronize()
    else:
        # CPU PATH
        a = np.random.rand(size, size)
        b = np.random.rand(size, size)
        np.dot(a, b)

    return float(time.time() - start)

def process_job(job):
    global last_work_time
    if not sio.connected: return False

    job_id = job['id']
    job_type = job['type']
    job_data = job['data']

    try:
        duration = 0
        if job_type == 'MATH_STRESS':
            duration = run_stress_test(job_data.get('iterations', 50000))
        elif job_type == 'MAT_MUL':
            duration = run_matrix_mul(job_data)

        last_work_time = time.time()

        if sio.connected:
            sio.emit('job:complete', {
                'chunkId': job_id,
                'result': 'Calculated',
                'durationMs': duration * 1000,
                'workerId': DEVICE_ID,
                'timestamp': time.time() * 1000
            })
            return True
    except Exception as e:
        print(f"\nJob Error: {e}")
        if sio.connected:
            sio.emit('job:complete', {
                'chunkId': job_id,
                'error': str(e),
                'workerId': DEVICE_ID
            })
        return False

# --- 5. EVENTS ---

@sio.event
def connect():
    print(f"\n✅ Connected! ID: {DEVICE_ID}")

    # Detect System Info
    import multiprocessing
    cores = multiprocessing.cpu_count()
    mem = 12

    sio.emit('device:register', {
        'name': DEVICE_NAME,
        'type': 'COLAB',
        'capabilities': {
            'cpuCores': cores,
            'memoryGB': mem,
            'gpuAvailable': HAS_GPU,
            # If GPU, we can handle huge concurrency because CUDA is parallel
            'maxConcurrency': cores * 4 if HAS_GPU else cores,
            'supportedJobs': ['MATH_STRESS', 'MAT_MUL']
        }
    })

@sio.on('job:batch')
def on_batch(jobs):
    global is_working
    is_working = True
    print(f"\r📦 Batch: {len(jobs)} | GPU: {'ON' if HAS_GPU else 'OFF'}", end="")

    for job in jobs:
        if not sio.connected: break
        process_job(job)

    is_working = False

    # Pull next batch
    if sio.connected:
        sio.emit('job:request_batch')

@sio.on('cmd:run_benchmark')
def on_benchmark():
    print("\n🚀 Starting Benchmark...", end="")
    try:
        start = time.time()
        score = 0

        if HAS_GPU:
            # CASE A: GPU (Heavy Matrix Mul)
            size = 1000
            a = torch.rand(size, size, device='cuda')
            b = torch.rand(size, size, device='cuda')
            torch.matmul(a, b)
            torch.cuda.synchronize()

            duration = time.time() - start
            # Formula: 2 Billion Ops / Seconds
            score = int(2000000000 / (duration + 0.00001))
            print(f" [GPU MODE] Score: {score:,}")

        else:
            # CASE B: CPU (Simple Loop)
            # 5 Million Iterations
            count = 5000000
            x = np.random.rand(int(count/100))
            np.sin(x) * np.sqrt(x)

            duration = time.time() - start
            # Formula: Iterations / Seconds
            score = int(count / (duration + 0.00001))
            print(f" [CPU MODE] Score: {score:,}")

        sio.emit('benchmark:result', {'score': score})

    except Exception as e:
        print(f" Benchmark Failed: {e}")

@sio.event
def disconnect():
    print("\n❌ Disconnected from server.")

# --- 6. BACKGROUND POLLER ---
def poller_loop():
    while True:
        try:
            # If idle for >1s, ask for work
            if sio.connected and not is_working:
                if time.time() - last_work_time > 1.0:
                    sio.emit('job:request_batch')
            time.sleep(1.0)
        except:
            pass


In [ ]:
# --- 3. IDENTITY ---
id_file = "device_identity.txt"
if os.path.exists(id_file):
    with open(id_file, "r") as f:
        DEVICE_ID = f.read().strip()
else:
    DEVICE_ID = f"colab-{str(uuid.uuid4())[:8]}"
    with open(id_file, "w") as f:
        f.write(DEVICE_ID)

sio = socketio.Client(reconnection=True, reconnection_delay=5)
is_working = False
last_work_time = time.time()


In [1]:
# --- 2. CONFIGURATION ---
SERVER_URL = "https://public-pride-bought-sys.trycloudflare.com/" # @param {type:"string"}
DEVICE_NAME = "Colab-Node-01" # @param {type:"string"}



# --- 7. MAIN START ---
def main():
    print(f"🚀 Initializing Worker {DEVICE_ID}...")

    # Start Poller
    t = threading.Thread(target=poller_loop, daemon=True)
    t.start()

    # Auth URL
    auth_url = f"{SERVER_URL}?persistentId={DEVICE_ID}"

    while True:
        try:
            if not sio.connected:
                print(f"Connecting to {SERVER_URL}...")
                sio.connect(auth_url, transports=['websocket', 'polling'])
                sio.wait()
        except KeyboardInterrupt:
            print("\nStopping...")
            break
        except Exception as e:
            print(f"Connection Error: {e}")
            time.sleep(5)


In [2]:
if __name__ == '__main__':
    main()

NameError: name 'main' is not defined